The idea is as follows:
- the agent finds its closest local waypoint. The local waypoints are only n-steps away from what ever position the agent is currently in 
- from this micro level waypoint the agent finds it's closest meso level waypoiny
- from the meso level waypoint the agent finds its macro level waypoint to travel from macro point to macro point

In [ ]:
'''
Imports external and own libraries
'''

import pickle
import math

import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader

from prettytable import PrettyTable

# own
import collector
import action
import world
import plot
import preprocess
import nets
import train

In [ ]:
env = world.init_env('MiniWorld-TMaze-v0')
plot.plot_obs_top_dep(env)
world.print_env_parameters(env)

In [ ]:
import random

amount_positions = 1
amount_trajectories = 100
length_trajectory = 5

action_sequences = []
for _ in range(amount_trajectories):
    seq = [random.randrange(0, 4, 1) for i in range(length_trajectory)]
    if seq not in action_sequences:
        action_sequences.append(seq)
        
positions = [[0, 0.0, 0]]
for _ in range(amount_positions-1):
    x = random.randrange(0, 110, 1)/10
    if x > 8.7:
        y = random.randrange(-74, 74, 1)/10
    else:
        y = random.randrange(-12, 15, 1)/10
    if [x, 0.0, y] not in positions:
        positions.append([x, 0.0, y])

reverse_action_sequences = []
for actions in action_sequences:
    reverse_action_sequences.append(actions + action.reverse_action_seq(actions))
    
print(reverse_action_sequences)
print(positions)

In [ ]:
env = world.init_env('MiniWorld-TMaze-v0')
for position in positions:
    for actions in reverse_action_sequences:
        env.place_entity(env.agent, pos=position)
        env.agent.dir = 0
        for action in actions:
                _, _, _, _ = env.step(action)
                position = env.agent.pos # coordinates
                angle = int(env.agent.dir * 180 / math.pi) % 360
                
                plot.plot_obs_top_dep(env)
                world.print_env_parameters(env)

In [ ]:
import random

def collect(env, amount_positions, amount_trajectories, length_trajectory, img_size=32, show=False,):
    '''
    Collect dataset containing actions, positions, angles, observations (first-
    person view), depth-view and the map.
    '''
    
    # generate random action sequences
    action_sequences = []
    for _ in range(amount_trajectories):
        seq = [random.randrange(0, 4, 1) for i in range(length_trajectory)]
        if seq not in action_sequences:
            action_sequences.append(seq)
    
    # generate random starting positions (within the T-maze)
    positions = [[0, 0.0, 0]]
    for _ in range(amount_positions):
        x = random.randrange(0, 110, 1)/10
        if x > 8.7:
            y = random.randrange(-74, 74, 1)/10
        else:
            y = random.randrange(-12, 15, 1)/10
        if [x, 0.0, y] not in positions:
            positions.append([x, 0.0, y])
            
    act_col = []
    pos_col = []
    angl_col = []
    obs_col = []
    top_col = []
    dep_col = []
    length_actions = len(actions)
    
    with tqdm(total=amount_positions*len(action_sequences), unit =" Steps", desc ="Progress") as pbar:
        for position in positions:
            for actions in action_sequences:
                env.place_entity(env.agent, pos=position)
                env.agent.dir = 0
                for action in actions:
                    _, _, _, _ = env.step(action)
                position = env.agent.pos # coordinates
                angle = int(env.agent.dir * 180 / math.pi) % 360

                observation = env.render('rgb_array')
                top_view = env.render('rgb_array', view='top') # map
                depth = env.render_depth()

                observation = resize(observation, (img_size, img_size))
                top_view = resize(top_view, (img_size, img_size))
                depth = resize(depth, (img_size, img_size))

                pos_col.append(position)
                angl_col.append(angle)
                obs_col.append(observation)
                top_col.append(top_view)
                dep_col.append(depth)

                pbar.update(1)   